In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd

In [ ]:
# Mounting Google Drive to read Numpy file 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loan and read Numpy file
coinbase = np.load('/content/coinbase_order_array.npy', allow_pickle=True)
coinbase

array([{'bids': [['0.02706', '0.3941605', 2]], 'asks': [['0.02707', '17.115', 2]], 'sequence': 2517907125, 'time': 1609195957834},
       {'bids': [['0.02706', '0.1191605', 1]], 'asks': [['0.02707', '3.075', 3]], 'sequence': 2517907249, 'time': 1609195959133},
       {'bids': [['0.02707', '2.8', 1]], 'asks': [['0.02708', '24.03321021', 4]], 'sequence': 2517907434, 'time': 1609195960437},
       ...,
       {'bids': [['0.0273', '11.82427217', 3]], 'asks': [['0.02731', '5.87095917', 2]], 'sequence': 2519953418, 'time': 1609239371302},
       {'bids': [['0.02729', '24.01996707', 7]], 'asks': [['0.0273', '5.87095917', 2]], 'sequence': 2519953656, 'time': 1609239373100},
       {'bids': [['0.02729', '29.53056707', 7]], 'asks': [['0.0273', '5.87095917', 2]], 'sequence': 2519953695, 'time': 1609239374755}],
      dtype=object)

In [ ]:
# Installing Python Spark
!pip install pyspark

     |████████████████████████████████| 204.2MB 64kB/s 
     |████████████████████████████████| 204kB 39.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=781a0d02e86fbb2982bed78b1a7b0bd99cb943034a6e5342c046aad8b0e19df8
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
# Importing necessary Spark libraries
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.ml.linalg import Vectors
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
#Making streaming for 10 seconds
sc = spark.sparkContext
ssc = StreamingContext(sc, 10)
rdd = ssc.sparkContext.parallelize(coinbase)

In [ ]:
def column(matrix, i):
    return [row[i] for row in matrix]

In [ ]:
#Mapping RDD 
rdd = rdd.map(lambda x: (x['time'], x['asks'], x['bids']))

In [ ]:
rdd.take(5)

[(1609195957834, [['0.02707', '17.115', 2]], [['0.02706', '0.3941605', 2]]),
 (1609195959133, [['0.02707', '3.075', 3]], [['0.02706', '0.1191605', 1]]),
 (1609195960437, [['0.02708', '24.03321021', 4]], [['0.02707', '2.8', 1]]),
 (1609195962189,
  [['0.02709', '22.27755749', 2]],
  [['0.02708', '4.84406447', 3]]),
 (1609195963505,
  [['0.02709', '22.27755749', 2]],
  [['0.02708', '4.84406447', 3]])]

In [ ]:
# Function to find max and min
def min_max(iterator):

   
    asks=[]
    for x in iterator:

      asks.append([x[0],x[1][0][0], x[2][0][0]])

    return asks

In [ ]:
final = rdd.mapPartitions(min_max)

In [ ]:
# Displaying the final result
final.take(5)

[[1609195957834, '0.02707', '0.02706'],
 [1609195959133, '0.02707', '0.02706'],
 [1609195960437, '0.02708', '0.02707'],
 [1609195962189, '0.02709', '0.02708'],
 [1609195963505, '0.02709', '0.02708']]